In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
from torch.utils.tensorboard import SummaryWriter
import random

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs, CovidDataset, Rescale, ToTensor
from utils import averageModels

In [ ]:
writer = SummaryWriter()

In [ ]:
dataset = CovidDataset(csv_path='./train.csv', transform=transforms.Compose([Rescale(64), ToTensor()]))
print(type(dataset))

In [ ]:
print(len(dataset))

In [ ]:
for i in range(len(dataset)):
    item = dataset.__getitem__(i)
    print("item ", i , " image shape= ", item['image'].shape)

In [ ]:
dset = getActualImgs(dataset, [0,1,2], 1)
print(type(dset))

In [ ]:
for batch_idx, (data, target) in enumerate(dset):
    print(data)

In [ ]:
import collections
a = np.random.choice(3, 2, replace=False)
b = np.random.choice(3, 2, replace=False)
c = np.random.choice(3, 2, replace=False)
print(a, b, c)
# collections.Counter(a)

In [ ]:
classes_indx = [i for i in range(3)]
print(classes_indx)

In [ ]:
# non iid data test
c_num = 3
num_users = 10
classes, images = c_num, int(300/c_num)
classes_indx = [i for i in range(classes)]
print("classes indx ", classes_indx)
users_dict = {i: np.array([]).astype(int) for i in range(num_users)}
print("users_dict", users_dict)
indeces = np.arange(classes*images)
unsorted_labels = np.random.choice(3, 300)

indeces_unsortedlabels = np.vstack((indeces, unsorted_labels))
print("indeces_unsortedlabels before shuffle: ", indeces_unsortedlabels)
shuffled_indices = np.random.permutation(len(indeces_unsortedlabels[0]))
indeces_unsortedlabels[0] = indeces_unsortedlabels[0][shuffled_indices]
indeces_unsortedlabels[1] = indeces_unsortedlabels[1][shuffled_indices]
print("indeces_unsortedlabels after shuffle: ", indeces_unsortedlabels)
indeces_labels = indeces_unsortedlabels[:, indeces_unsortedlabels[1, :].argsort()]
indeces = indeces_labels[0, :]
indeces_labels.astype(int)
indeces.astype(int)

index_label = [[] for i in range(c_num)]
for i in range(len(indeces_labels[1])):
    index_label[indeces_labels[1][i]].append(indeces_labels[0][i])
print(index_label)

In [ ]:
from itertools import combinations
client_classes = []
comb = []
for i in list(combinations(list(range(0,c_num)), 2)):
    print(i)
    comb.append(i)
print(comb)

# classes of client
for i in range(num_users):
    client_classes.append(comb[i%c_num])
client_classes = np.array(client_classes)
c = client_classes.flatten()
print(client_classes)

# count of labels
label_count = collections.Counter(c)
print(label_count)

In [ ]:
indeces = [int(i) for i in indeces]
print(indeces)
for i in range(num_users):
    np.random.seed(i)
    temp = set(np.random.choice(classes_indx, 2, replace=False))
    print(temp)
    classes_indx = list(set(classes_indx) - temp)
    for t in temp:
        users_dict[i] = np.concatenate((users_dict[i], indeces[t*images:(t+1)*images]), axis=0)